In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/bike-sharing-demand/sampleSubmission.csv
/kaggle/input/bike-sharing-demand/train.csv
/kaggle/input/bike-sharing-demand/test.csv


In [2]:
import pandas as pd
import numpy as np
import seaborn as sns 
import matplotlib.pyplot as plt
import xgboost as xgb
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score

%matplotlib inline 

In [3]:
def get_lag_time(df,col,n_back):
    for i in range(n_back):
        df[col+'-'+str(i+1)] = df[col].shift(i+1)
    return df

In [4]:
def get_lead_time(df,col,n_back):
    for i in range(n_back):
        df[col+'+'+str(i+1)] = df[col].shift(-(i+1))
    return df

In [5]:
def read_edit(path):    
        df = pd.read_csv(path)

        # converting date to date time
        df['datetime']=pd.to_datetime(df['datetime'])
        df['date'] = pd.to_datetime(df['datetime']).dt.date
        df['Time'] = pd.to_datetime(df['datetime']).dt.time
        df['hour'] = pd.to_datetime(df['datetime']).dt.hour

        # making date and night out of hours
        df['label_day_night']=df['hour'].apply(lambda x : 'Night' if (x >20 or x<5) else( 'Day'))
        # getting day names
        df['week_day']=df['datetime'].dt.day_name()
        # getting months
        df['month']=df['datetime'].dt.month
        # Encoding days of the week
        mapping_dictDay={'Monday':1,'Tuesday':2,'Wednesday':3,'Thursday':4,'Friday':5,'Saturday':6,'Sunday':7}
        df['week_day_encoding']=df['week_day'].map(mapping_dictDay)

        # getting dummies for seasons,holiday,functioning_day

        df['date'], Categorical= pd.factorize(df['date'])
        df['week_day'], Categorical= pd.factorize(df['week_day'])

        # applying lag_time & lead_time
        df = get_lag_time(df,'windspeed',2)
        df = get_lead_time(df,'temp',1)
        

        return df

In [6]:
# reading train and test
train= read_edit('/kaggle/input/bike-sharing-demand/train.csv')
test = read_edit('/kaggle/input/bike-sharing-demand/test.csv')

test = test.drop(['date','datetime','Time','atemp','label_day_night','week_day'],axis =1)
X=train.drop(['count','datetime','Time','date','atemp','label_day_night','week_day'],axis =1)
y=train['count'] #the target

X.head()

,season,holiday,workingday,weather,temp,humidity,windspeed,casual,registered,hour,month,week_day_encoding,windspeed-1,windspeed-2,temp+1
0,1,0,0,1,9.84,81,0.0,3,13,0,1,6,NaN,NaN,9.02
1,1,0,0,1,9.02,80,0.0,8,32,1,1,6,0.0,NaN,9.02
2,1,0,0,1,9.02,80,0.0,5,27,2,1,6,0.0,0.0,9.84
3,1,0,0,1,9.84,75,0.0,3,10,3,1,6,0.0,0.0,9.84
4,1,0,0,1,9.84,75,0.0,0,1,4,1,6,0.0,0.0,9.84


In [7]:
# train - test split
X_train,X_val,y_train,y_val=train_test_split(X,y,test_size=0.1, random_state=42)
# xgboost model
xg_reg = xgb.XGBRegressor(objective='count:poisson',random_state=42,colsample_bytree = 0.6, learning_rate = 0.1,max_depth=5
                 ,alpha = 10, n_estimators = 1000)

In [8]:
from sklearn.metrics import mean_squared_log_error
# fit the model
xg_reg=xg_reg.fit(X_train,y_train)
#==========================================================
#========================train=============================
y_pred=xg_reg.predict(X_train)
print('Accuracy of XGB regressor on train set: {:.5f}'.format(xg_reg.score(X_train, y_train)))
rmsle_train = np.sqrt(mean_squared_log_error(y_train, y_pred))
print("RMSE-train: %f" % (rmsle_train))


#==========================================================
#======================validation==========================

y_pred1=xg_reg.predict(X_val)
print('Accuracy of XGB regressor on test set: {:.5f}'.format(xg_reg.score(X_val, y_val)))
rmsle = np.sqrt(mean_squared_log_error(y_val, y_pred1))
print("RMSE-val: %f" % (rmsle))
#==========================================================
#====================cross-val=============================
scores = cross_val_score(xg_reg, X_train, y_train, cv=5)
print(scores)

Accuracy of XGB regressor on train set: 0.99968
RMSE-train: 0.026020
Accuracy of XGB regressor on test set: 0.99892
RMSE-val: 0.038262
[0.99840008 0.99873952 0.99871685 0.99831015 0.99833005]
